In [404]:
p_storage_dir = "../../storage"
p_notebooks_root = None

In [ ]:
if p_notebooks_root is not None:
    import os
    os.chdir(p_notebooks_root)

In [405]:
import numpy as np
import pandas as pd

import re
from pathlib import Path
import unicodedata

IFG_PRODUZ_STORAGE_DIR = Path(f"{p_storage_dir}/ifg_produz")

# Projetos de Pesquisa

## Limpezas básicas

In [406]:
projetos_pesquisa = pd.read_csv(IFG_PRODUZ_STORAGE_DIR / "extracted/projetos_pesquisa.csv", delimiter=";")

projetos_pesquisa.sample(n=5)

,id_projeto_pesquisa,nome_projeto,descricao_projeto,data_inicio,data_termino,financiador_projeto,instituicao_financiadora,orgao_do_projeto,id_curriculo_id
153,150,O clima urbano de Jataí utilizando medidas de ...,NaN,2011,2012.0,-NAO_INFORMADO,-,Departamento de Geografia,52
1416,1417,Criação e implementação de ambientes de formaç...,Objetiva a melhoria dos processos de ensino-ap...,2007,2008.0,-BOLSA,-Financiadora de Estudos e Projetos,"Pró-reitoria de Extensão, Cultura e Assuntos E...",697
2235,2247,Competência Linguística em Língua Inglesa e a ...,A internacionalização do mercado de trabalho p...,2016,2017.0,NaN,NaN,Instituto Federal de Goiás - Câmpus Uruaçu,1060
1962,1975,Análise da distribuição geográfica e caracteri...,O estudo sobre o patrimônio cultural material ...,2019,2020.0,-BOLSA,-Conselho Nacional de Desenvolvimento Científi...,IFG-Câmpus Anápolis,949
2951,2976,Desenvolvimento de aparato e metodologia para ...,Neste trabalho é desenvolvido aparato eletrôni...,2014,2016.0,NaN,NaN,Campus Goiânia,1385


In [407]:
projetos_pesquisa['data_termino'] = projetos_pesquisa['data_termino'].astype('Int64')

In [408]:
projetos_pesquisa = projetos_pesquisa.rename(columns={
    'id_projeto_pesquisa': 'codigo',
    'nome_projeto': 'nome',
    'descricao_projeto': 'descricao',
    'data_inicio': 'ano_inicio',
    'data_termino': 'ano_termino',
    'orgao_do_projeto': 'orgao',
    'id_curriculo_id': 'codigo_curriculo'
})

In [409]:
projetos_pesquisa.columns

Index(['codigo', 'nome', 'descricao', 'ano_inicio', 'ano_termino',
       'financiador_projeto', 'instituicao_financiadora', 'orgao',
       'codigo_curriculo'],
      dtype='object')

In [410]:
projetos_pesquisa['financiador_projeto'].unique()

array(['-NAO_INFORMADO', nan, '-AUXILIO_FINANCEIRO', '-BOLSA',
       '-REMUNERACAO', '-BOLSA-AUXILIO_FINANCEIRO',
       '-AUXILIO_FINANCEIRO-BOLSA', '-OUTRO', '-COOPERACAO',
       '-COOPERACAO-AUXILIO_FINANCEIRO', '-AUXILIO_FINANCEIRO-COOPERACAO',
       '-AUXILIO_FINANCEIRO-BOLSA-COOPERACAO',
       '-COOPERACAO-COOPERACAO-AUXILIO_FINANCEIRO',
       '-AUXILIO_FINANCEIRO-COOPERACAO-COOPERACAO',
       '-BOLSA-COOPERACAO-AUXILIO_FINANCEIRO', '-BOLSA-BOLSA',
       '-OUTRO-AUXILIO_FINANCEIRO',
       '-BOLSA-AUXILIO_FINANCEIRO-AUXILIO_FINANCEIRO',
       '-AUXILIO_FINANCEIRO-REMUNERACAO-BOLSA',
       '-AUXILIO_FINANCEIRO-AUXILIO_FINANCEIRO-BOLSA',
       '-REMUNERACAO-BOLSA', '-BOLSA-REMUNERACAO',
       '-AUXILIO_FINANCEIRO-REMUNERACAO-REMUNERACAO',
       '-AUXILIO_FINANCEIRO-AUXILIO_FINANCEIRO', '-COOPERACAO-BOLSA',
       '-AUXILIO_FINANCEIRO-BOLSA-AUXILIO_FINANCEIRO',
       '-REMUNERACAO-COOPERACAO', '-BOLSA-AUXILIO_FINANCEIRO-BOLSA-BOLSA',
       '-BOLSA-COOPERACAO', '-AUXILI

In [411]:
projetos_pesquisa['instituicao_financiadora'].unique()

array(['-', nan,
       '-Conselho Nacional de Desenvolvimento Científico e Tecnológico',
       '-Coordenação de Aperfeiçoamento de Pessoal de Nível Superior',
       '-Universidade Estadual de Santa Cruz',
       '-Fundação de Amparo a Pesquisa do Estado de Goiás',
       '-Instituto Federal de Goiás', '-Cyclades Brasil',
       '-Fundação de Desenvolvimento da Pesquisa',
       '-Natura Cosméticos SA-Fundação Arthur Bernardes',
       '-Empresa Brasileira de Pesquisa Agropecuária',
       '-Instituto Federal de Educação, Ciência e Tecnologia de Goiás',
       '-Instituto Federal de Educação, Ciência e Tecnologia de Goiás-Conselho Nacional de Desenvolvimento Científico e Tecnológico',
       '-Furnas Centrais Elétricas S.A.-Agência Nacional de Energia Elétrica',
       '-Agência Nacional de Energia Elétrica-Furnas Centrais Elétricas S.A.',
       '-Conselho Nacional de Desenvolvimento Científico e Tecnológico-Coordenação de Aperfeiçoamento de Pessoal de Nível Superior-Fundação de Amp

In [412]:
projetos_pesquisa[['financiador_projeto', 'instituicao_financiadora']] = projetos_pesquisa[['financiador_projeto', 'instituicao_financiadora']].replace('^-{1}', '', regex=True)

In [413]:
projetos_pesquisa['financiador_projeto'] = projetos_pesquisa['financiador_projeto'].replace('NAO_INFORMADO', np.nan)

In [414]:
projetos_pesquisa['instituicao_financiadora'] = projetos_pesquisa['instituicao_financiadora'].replace('', np.nan)

## Colocar a sigla do câmpus nos órgãos que se referem às unidades do IFG

In [415]:
list(projetos_pesquisa['orgao'].unique())

['Escola de Engenharia Elétrica',
 'Centro de Inovação Tecnológica',
 'Instituto Federal de Goiás - Câmpus Goiânia',
 'Programa de Bolsas de Iniciação Científica da UESC - ICB',
 'Campus de Jataí',
 'Secretaria Municipal de Meio Ambiente, Ciência e Tecnologia de Jataí',
 'Faculdade de Enfermagem e Nutrição',
 'Instituto Federal de Goiás - Campus Águas Lindas de Goiás',
 'Campus Goiânia',
 'Conselho de Ensino, Pesquisa e Extensão',
 'Campus Inhumas',
 'IPEA',
 'Faculdade de Administração, Ciências Contábeis e Ciências Econômicas',
 'Centro de Estudos Gerais',
 'Coordenação de História da Ciência e Tecnologia',
 'Instituto Federal de Educação, Ciência e Tecnologia de Goiás/Câmpus Jataí',
 'Faculdade de Letras',
 'Campus Uruaçu',
 'Goiânia',
 'Câmpus Inhumas',
 'Escola de Engenharia Elétrica, Mecânica e de Computação',
 'Mestrado Profissionalizante em Educação para Ciências e Matemática',
 'Centro Federal de Goiás - Cefet - Unidade Jataí',
 'Instituto Federal de Educação, Ciência e Tecnol

In [416]:
projetos_pesquisa['sigla_orgao'] = pd.Series([None] * len(projetos_pesquisa), dtype='object')

In [417]:
def normalizar_string(s):
    if pd.notna(s):
        # Transforma todos os caracteres em minúsculas
        s = s.lower()
        
        # Remove acentos das letras
        s = unicodedata.normalize("NFD", s).encode("ascii", "ignore").decode("utf-8")
    
        # Substitui todos os caracteres que não são letras por um espaço em branco
        s = re.sub(r'[^A-Za-z]', ' ', s)
    
        # Substitui espaços consecutivos por apenas um espaço
        s = re.sub(r'\s{2,}', ' ', s)
    
        # Remove os espaços em branco do começo e fim
        s = s.strip() 
    
    return s

In [418]:
projetos_pesquisa['orgao_normalizado'] = projetos_pesquisa['orgao'].apply(normalizar_string)

In [419]:
projetos_pesquisa[(projetos_pesquisa['orgao_normalizado'].notna()) & ((projetos_pesquisa['orgao_normalizado'].str.contains('uruacu')))]['orgao_normalizado'].unique()

array(['campus uruacu', 'unidade universitaria de uruacu', 'uruacu',
       'instituto federal de educacao ciencia e tecnologia de goias campus uruacu',
       'ifg campus uruacu',
       'instituto federal de educacao ciencia e tecnologia de goias uruacu',
       'gerencia de pesquisa extensao e pos graduacao do ifg campus uruacu',
       'instituto federal de goias campus uruacu',
       'coordenacao das areas academicas campus uruacu', 'ifg uruacu',
       'gepex campus uruacu'], dtype=object)

### Fazer uma bag of words geral

Essa técnica é utilizada para investigar quais são as palavras que mais se repetem em um conjunto de palavras. No nosso caso será utilizada para destacarmos os termos principais que podem indicar a referência de um órgão a uma unidade do IFG.

In [420]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords

In [421]:
nltk.download('popular')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /home/marcel/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /home/marcel/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /home/marcel/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /home/marcel/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /home/marcel/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /home/marcel/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to

True

In [422]:
corpus = ' '.join(projetos_pesquisa['orgao_normalizado'].dropna())

In [423]:
words = [word for word in word_tokenize(corpus) if word not in stopwords.words('portuguese')]

In [424]:
bag_geral = FreqDist(words)

In [425]:
bag_geral.pprint(30)

FreqDist({'campus': 1809, 'instituto': 999, 'goias': 802, 'federal': 736, 'goiania': 596, 'ciencias': 362, 'educacao': 345, 'ifg': 308, 'faculdade': 299, 'tecnologia': 248, 'inhumas': 235, 'jatai': 232, 'departamento': 224, 'pesquisa': 193, 'centro': 178, 'ciencia': 171, 'anapolis': 167, 'escola': 163, 'itumbiara': 158, 'reitoria': 156, 'graduacao': 154, 'pos': 151, 'pro': 127, 'engenharia': 124, 'aparecida': 117, 'fisica': 106, 'humanas': 105, 'luziania': 102, 'uruacu': 95, 'formosa': 92, ...})


### Algoritmo para classificar se o órgão se referencia, ou não, a uma unidade do IFG

In [426]:
def link_orgao_to_campus(row, sigla, additional_tokens=None, blockant_tokens=None, debug=False):
    from thefuzz import fuzz
    from math import ceil
    import re

    sigla_to_campus_name = {
        'LIN': 'aguas lindas',
        'ANA': 'anapolis',
        'APA': 'aparecida de goiania',
        'GOI': 'cidade de goias',
        'FOR': 'formosa',
        'GYN': 'goiania',
        'OES': 'goiania oeste',
        'INH': 'inhumas',
        'ITU': 'itumbiara',
        'JAT': 'jatai',
        'LUZ': 'luziania',
        'SEN': 'senador canedo',
        'URU': 'uruacu',
        'VAL': 'valparaiso',
        'REI': 'reitoria'
    }

    try:
        campus_name = sigla_to_campus_name[sigla]
    except KeyError:
        raise KeyError(f'Sigla inválida, precisa ser alguma das seguintes: {list(sigla_to_campus_name.keys())}')

    row['sigla_orgao'] = None
    
    base_tokens = ['campus', 'instituto', 'federal', 'ifg']

    tokens = base_tokens

    orgao_normalizado = ' '.join(set([word for word in row['orgao_normalizado'].split(' ') if word not in stopwords.words('portuguese')]))
    orgao_normalizado_sem_cidade = ' '.join([word for word in orgao_normalizado.split(' ') if word != campus_name])

    if blockant_tokens is not None and isinstance(blockant_tokens, list) and len(blockant_tokens) > 0:
        for blockant_token in blockant_tokens:
            if re.search(blockant_token, orgao_normalizado) is not None or fuzz.partial_ratio(blockant_token, orgao_normalizado) >= 85:
                return row
    
    if additional_tokens is not None and isinstance(additional_tokens, list) and len(additional_tokens) > 0:
        tokens += additional_tokens
    
    results = []
    
    for token_to_compare in tokens:
        results.append(fuzz.partial_ratio(token_to_compare, orgao_normalizado_sem_cidade))

    max_score = max(results)

    amount_tokens_matched = len(list(filter(lambda result: result >= 80, results)))
    amount_tokens_orgao_normalizado_sem_cidade = len(orgao_normalizado_sem_cidade.split(' '))
    orgao_string_coverage = ceil((amount_tokens_matched / amount_tokens_orgao_normalizado_sem_cidade) * 100)

    orgao_string_coverage_threshold = None

    if amount_tokens_orgao_normalizado_sem_cidade <= 3:
        orgao_string_coverage_threshold = 25
    elif amount_tokens_orgao_normalizado_sem_cidade <= 5:
        orgao_string_coverage_threshold = 20
    elif amount_tokens_orgao_normalizado_sem_cidade > 5:
        orgao_string_coverage_threshold = 15
    
    if max_score >= 80 and orgao_string_coverage >= orgao_string_coverage_threshold:
        row['sigla_orgao'] = sigla
    
    if debug:
        print('-' * 8)
        print(f'Nome órgão normalizado: {orgao_normalizado}')
        print(f'Nome órgão normalizado sem cidade: {orgao_normalizado_sem_cidade}')
        print(f'Resultados: {results}')
        print(f'String coverage: {orgao_string_coverage}')
        print(f'Pontuação máxima: {max_score}')
        print('-' * 8)

    return row

### Funções auxiliares

In [427]:
def get_orgao_normalizado_by_contains_term(term):
    return projetos_pesquisa[(projetos_pesquisa['orgao_normalizado'].notna()) & ((projetos_pesquisa['orgao_normalizado'].str.contains(term)))]

In [428]:
def get_orgao_normalizado_corpus(search_term):
    return ' '.join(get_orgao_normalizado_by_contains_term(search_term)['orgao_normalizado'])

In [429]:
def make_bag_from_corpus(corpus, tokens_to_ignore):
    if isinstance(tokens_to_ignore, str):
        tokens_to_ignore = [tokens_to_ignore]
    
    return FreqDist([word for word in corpus.split(' ') if word not in tokens_to_ignore and word not in stopwords.words('portuguese')])

In [430]:
def get_notna_orgao_normalizado_indexes(search_term):
    return get_orgao_normalizado_by_contains_term(search_term).index

### Aplicação do algoritmo

#### Jataí

In [431]:
corpus_jatai = get_orgao_normalizado_corpus('jatai')

In [432]:
bag_jatai = make_bag_from_corpus(corpus_jatai, 'jatai')

In [433]:
bag_jatai.pformat(20)

"FreqDist({'uruacu': 95, 'campus': 85, 'instituto': 34, 'federal': 34, 'goias': 34, 'ifg': 8, 'educacao': 4, 'ciencia': 4, 'tecnologia': 4, 'unidade': 2, 'universitaria': 2, 'gerencia': 2, 'pesquisa': 2, 'extensao': 2, 'pos': 2, 'graduacao': 2, 'coordenacao': 1, 'areas': 1, 'academicas': 1, 'gepex': 1})"

In [434]:
jatai_indexes = get_notna_orgao_normalizado_indexes('jatai')

projetos_pesquisa.loc[jatai_indexes] = projetos_pesquisa.loc[jatai_indexes].apply(
    link_orgao_to_campus,
    args=('JAT', ['uned', 'cefet', 'centro ensino'], ['avancado']),
    axis=1
)

#### Uruaçu

In [435]:
corpus_uruacu = get_orgao_normalizado_corpus('uruacu')

In [436]:
bag_uruacu = make_bag_from_corpus(corpus_uruacu, 'uruacu')

In [437]:
bag_uruacu.pformat(20)

"FreqDist({'campus': 85, 'instituto': 34, 'federal': 34, 'goias': 34, 'ifg': 8, 'educacao': 4, 'ciencia': 4, 'tecnologia': 4, 'unidade': 2, 'universitaria': 2, 'gerencia': 2, 'pesquisa': 2, 'extensao': 2, 'pos': 2, 'graduacao': 2, 'coordenacao': 1, 'areas': 1, 'academicas': 1, 'gepex': 1})"

In [438]:
uruacu_indexes = get_notna_orgao_normalizado_indexes('uruacu')

projetos_pesquisa.loc[uruacu_indexes] = projetos_pesquisa.loc[uruacu_indexes].apply(
    link_orgao_to_campus,
    args=('URU',),
    axis=1
)

In [439]:
temp = projetos_pesquisa.loc[uruacu_indexes]

temp[temp['sigla_orgao'].notna()]['orgao_normalizado'].unique()

array(['campus uruacu',
       'instituto federal de educacao ciencia e tecnologia de goias campus uruacu',
       'ifg campus uruacu',
       'instituto federal de educacao ciencia e tecnologia de goias uruacu',
       'gerencia de pesquisa extensao e pos graduacao do ifg campus uruacu',
       'instituto federal de goias campus uruacu',
       'coordenacao das areas academicas campus uruacu', 'ifg uruacu',
       'gepex campus uruacu'], dtype=object)

#### Anápolis

In [440]:
corpus_anapolis = get_orgao_normalizado_corpus('anapolis')

In [441]:
bag_anapolis = make_bag_from_corpus(corpus_anapolis, 'anapolis')

In [442]:
bag_anapolis.pformat(20)

"FreqDist({'campus': 141, 'ifg': 33, 'instituto': 31, 'federal': 31, 'goias': 31, 'ciencia': 11, 'tecnologia': 11, 'educacao': 8, 'ciencias': 5, 'departamento': 4, 'areas': 4, 'academicas': 4, 'unidade': 3, 'exatas': 3, 'tecnologicas': 3, 'centro': 3, 'universitario': 3, 'gepex': 2, 'universitaria': 2, 'humanas': 2, ...})"

In [443]:
anapolis_indexes = get_notna_orgao_normalizado_indexes('anapolis')

projetos_pesquisa.loc[anapolis_indexes] = projetos_pesquisa.loc[anapolis_indexes].apply(
    link_orgao_to_campus,
    args=('ANA', None, ['ccseh', 'ciencias']),
    axis=1
)

In [444]:
temp = projetos_pesquisa.loc[anapolis_indexes]

temp[temp['sigla_orgao'].notna()]['orgao_normalizado'].unique()

array(['campus anapolis ifg', 'gepex campus anapolis', 'campus anapolis',
       'ifg campus anapolis',
       'instituto federal de goias campus anapolis',
       'instituto federal de goias campus anapolis go',
       'campus de anapolis',
       'departamento de areas academicas campus anapolis'], dtype=object)

#### Inhumas

In [445]:
corpus_inhumas = get_orgao_normalizado_corpus('inhumas')

In [446]:
bag_inhumas = make_bag_from_corpus(corpus_inhumas, 'inhumas')

In [447]:
bag_inhumas.pformat(30)

"FreqDist({'campus': 220, 'federal': 32, 'goias': 31, 'instituto': 30, 'ifg': 11, 'unidade': 7, 'ensino': 7, 'campi': 3, 'descentralizada': 3, 'faculdade': 2, 'fac': 2, 'goiania': 2, 'jatai': 2, 'educacao': 2, 'ciencia': 2, 'tecnologia': 2, 'nstituto': 2, 'depto': 1, 'areas': 1, 'academicas': 1, 'go': 1})"

In [448]:
inhumas_indexes = get_notna_orgao_normalizado_indexes('inhumas')

projetos_pesquisa.loc[inhumas_indexes] = projetos_pesquisa.loc[inhumas_indexes].apply(
    link_orgao_to_campus,
    args=('INH',),
    axis=1
)

In [449]:
temp = projetos_pesquisa.loc[inhumas_indexes]

temp[temp['sigla_orgao'].notna()]['orgao_normalizado'].unique()

array(['campus inhumas', 'campus de inhumas',
       'instituto federal de goias campus inhumas', 'ifg campus inhumas',
       'campus goiania campus inhumas campus jatai',
       'campus inhumas depto de areas academicas', 'ifg campi inhumas',
       'instituto federal de goias campus inhumas go',
       'campus inhumas ifg', 'campus inhumas goiania e jatai',
       'instituto federal de educacao ciencia e tecnologia de goias campus inhumas',
       'nstituto federal de goias campus inhumas',
       'nstituto federal de educacao ciencia e tecnologia campus inhumas'],
      dtype=object)

#### Itumbiara

In [450]:
corpus_itumbiara = get_orgao_normalizado_corpus('itumbiara')

In [451]:
bag_itumbiara = make_bag_from_corpus(corpus_itumbiara, 'itumbiara')

In [452]:
bag_itumbiara.pformat(30)

"FreqDist({'campus': 113, 'instituto': 21, 'ifg': 21, 'federal': 16, 'goias': 16, 'educacao': 12, 'ifgoias': 11, 'ciencia': 9, 'tecnologia': 9, 'luterano': 7, 'ensino': 7, 'superior': 7, 'subsecretaria': 3, 'regional': 3, 'unidade': 3, 'universitaria': 3, 'institutol': 2})"

In [453]:
itumbiara_indexes = get_notna_orgao_normalizado_indexes('itumbiara')

projetos_pesquisa.loc[itumbiara_indexes] = projetos_pesquisa.loc[itumbiara_indexes].apply(
    link_orgao_to_campus,
    args=('ITU', None, ['luterano']),
    axis=1
)

In [454]:
temp = projetos_pesquisa.loc[itumbiara_indexes]

temp[temp['sigla_orgao'].isna()]['orgao_normalizado'].unique()

array(['itumbiara', 'subsecretaria regional de educacao de itumbiara',
       'instituto luterano de ensino superior de itumbiara',
       'unidade universitaria de itumbiara'], dtype=object)

#### Luziânia

In [455]:
corpus_luziania = get_orgao_normalizado_corpus('luziania')

In [456]:
bag_luziania = make_bag_from_corpus(corpus_luziania, 'luziania')

In [457]:
bag_luziania.pformat(30)

"FreqDist({'campus': 97, 'departamento': 14, 'assuntos': 14, 'academicos': 14, 'ifg': 9, 'instituto': 9, 'federal': 9, 'goias': 9, 'go': 4, 'campi': 2, 'gerencia': 1, 'ensino': 1, 'pesquisa': 1, 'extensao': 1})"

In [458]:
luziania_indexes = get_notna_orgao_normalizado_indexes('luziania')

projetos_pesquisa.loc[luziania_indexes] = projetos_pesquisa.loc[luziania_indexes].apply(
    link_orgao_to_campus,
    args=('LUZ',),
    axis=1
)

In [459]:
temp = projetos_pesquisa.loc[luziania_indexes]

temp[temp['sigla_orgao'].notna()]['orgao_normalizado'].unique()

array(['campus luziania', 'ifg luziania',
       'instituto federal de goias campus luziania',
       'gerencia ensino pesquisa e extensao do ifg luziania',
       'ifg campus luziania',
       'departamento de assuntos academicos campus luziania',
       'instituto federal de goias campus luziania go', 'campi luziania'],
      dtype=object)

#### Valparaíso

In [460]:
corpus_valparaiso = get_orgao_normalizado_corpus('valparaiso')

In [461]:
bag_valparaiso = make_bag_from_corpus(corpus_valparaiso, 'valparaiso')

In [462]:
bag_valparaiso.pformat(30)

"FreqDist({'campus': 32, 'goias': 25, 'ifg': 11, 'instituto': 6, 'federal': 6, 'departamento': 4, 'areas': 4, 'academicas': 4, 'gepex': 3, 'go': 1, 'colegio': 1, 'educacao': 1, 'ciencia': 1, 'tecnologia': 1})"

In [463]:
valparaiso_indexes = get_notna_orgao_normalizado_indexes('valparaiso')

projetos_pesquisa.loc[valparaiso_indexes] = projetos_pesquisa.loc[valparaiso_indexes].apply(
    link_orgao_to_campus,
    args=('VAL',),
    axis=1
)

In [464]:
temp = projetos_pesquisa.loc[valparaiso_indexes]

temp[temp['sigla_orgao'].notna()]['orgao_normalizado'].unique()

array(['campus valparaiso',
       'instituto federal de goias campus valparaiso go',
       'ifg campus valparaiso',
       'instituto federal de goias campus valparaiso de goias',
       'instituto federal de goias campus valparaiso',
       'departamento de areas academicas do campus de valparaiso de goias',
       'gepex ifg valparaiso de goias',
       'instituto federal de goias ifg campus valparaiso',
       'campus valparaiso de goias', 'ifg campus valparaiso de goias',
       'instituto federal de educacao ciencia e tecnologia de goias valparaiso'],
      dtype=object)

#### Senador Canedo

In [465]:
corpus_senador_canedo = get_orgao_normalizado_corpus('senador canedo')

In [466]:
bag_senador_canedo = make_bag_from_corpus(corpus_senador_canedo, ['senador', 'canedo'])

In [467]:
bag_senador_canedo.pformat(30)

"FreqDist({'campus': 49, 'instituto': 29, 'federal': 29, 'goias': 28, 'go': 7, 'ifg': 4, 'educacao': 3, 'campi': 1})"

In [468]:
senador_canedo_indexes = get_notna_orgao_normalizado_indexes('senador canedo')

projetos_pesquisa.loc[senador_canedo_indexes] = projetos_pesquisa.loc[senador_canedo_indexes].apply(
    link_orgao_to_campus,
    args=('SEN',),
    axis=1
)

In [469]:
temp = projetos_pesquisa.loc[senador_canedo_indexes]

temp[temp['sigla_orgao'].notna()]['orgao_normalizado'].unique()

array(['ifg campus senador canedo',
       'instituto federal de goias campus senador canedo',
       'instituto federal de goias senador canedo',
       'campus senador canedo',
       'instituto federal de goias campus senador canedo go',
       'instituto federal de educacao campi senador canedo',
       'instituto federal de educacao de goias campus senador canedo go'],
      dtype=object)

#### Águas Lindas

In [470]:
corpus_aguas_lindas = get_orgao_normalizado_corpus('aguas lindas')

In [471]:
bag_aguas_lindas = make_bag_from_corpus(corpus_aguas_lindas, ['aguas', 'lindas'])

In [472]:
bag_aguas_lindas.pformat(30)

"FreqDist({'campus': 39, 'goias': 30, 'instituto': 20, 'federal': 20, 'ifg': 2, 'goiania': 1, 'oeste': 1})"

In [473]:
aguas_lindas_indexes = get_notna_orgao_normalizado_indexes('aguas lindas')

projetos_pesquisa.loc[aguas_lindas_indexes] = projetos_pesquisa.loc[aguas_lindas_indexes].apply(
    link_orgao_to_campus,
    args=('LIN',),
    axis=1
)

In [474]:
temp = projetos_pesquisa.loc[aguas_lindas_indexes]

temp[temp['sigla_orgao'].notna()]['orgao_normalizado'].unique()

array(['instituto federal de goias campus aguas lindas de goias',
       'instituto federal de goias campus aguas lindas',
       'ifg campus aguas lindas', 'campus aguas lindas',
       'instituto federal de goias ifg campus aguas lindas',
       'campus aguas lindas e goiania oeste'], dtype=object)

#### Cidade de Goiás

In [475]:
corpus_cidade_goias = get_orgao_normalizado_corpus('cidade de goias')

In [476]:
bag_cidade_goias = make_bag_from_corpus(corpus_cidade_goias, ['cidade', 'goias'])

In [477]:
bag_cidade_goias.pformat(30)

"FreqDist({'campus': 57, 'federal': 13, 'instituto': 9, 'ifg': 7, 'institudo': 4, 'departamento': 1, 'academico': 1, 'cora': 1, 'coralina': 1, 'sede': 1})"

In [478]:
cidade_goias_indexes = get_notna_orgao_normalizado_indexes('cidade de goias')

projetos_pesquisa.loc[cidade_goias_indexes] = projetos_pesquisa.loc[cidade_goias_indexes].apply(
    link_orgao_to_campus,
    args=('GOI', None, ['coralina']),
    axis=1
)

In [479]:
temp = projetos_pesquisa.loc[cidade_goias_indexes]

temp[temp['sigla_orgao'].notna()]['orgao_normalizado'].unique()

array(['campus cidade de goias',
       'institudo federal de goias campus cidade de goias',
       'ifg campus cidade de goias',
       'instituto federal de goias campus cidade de goias',
       'departamento academico campus cidade de goias'], dtype=object)

#### Formosa

In [480]:
corpus_formosa = get_orgao_normalizado_corpus('formosa')

In [481]:
bag_formosa = make_bag_from_corpus(corpus_formosa, 'formosa')

In [482]:
bag_formosa.pformat(30)

"FreqDist({'campus': 79, 'instituto': 26, 'federal': 26, 'goias': 24, 'unidade': 5, 'educacao': 4, 'ifg': 4, 'ciencia': 3, 'tecnologia': 3, 'departamento': 3, 'areas': 3, 'academicas': 3, 'camous': 2, 'go': 2, 'brasil': 1})"

In [483]:
formosa_indexes = get_notna_orgao_normalizado_indexes('formosa')

projetos_pesquisa.loc[formosa_indexes] = projetos_pesquisa.loc[formosa_indexes].apply(
    link_orgao_to_campus,
    args=('FOR', ['unidade']),
    axis=1
)

In [484]:
temp = projetos_pesquisa.loc[formosa_indexes]

temp[temp['sigla_orgao'].notna()]['orgao_normalizado'].unique()

array(['instituto federal de educacao ciencia e tecnologia de goias campus formosa',
       'campus formosa', 'instituto federal de goias campus formosa',
       'camous formosa',
       'instituto federal de educacao do goias campus formosa go brasil',
       'ifg campus formosa',
       'instituto federal de educacao ciencia e tecnologia campus formosa',
       'unidade formosa',
       'departamento de areas academicas campus formosa',
       'campus formosa go'], dtype=object)

#### Aparecida de Goiânia

In [485]:
corpus_aparecida = get_orgao_normalizado_corpus('aparecida de goiania')

In [486]:
bag_aparecida = make_bag_from_corpus(corpus_aparecida, ['aparecida', 'goiania'])

In [487]:
bag_aparecida.pformat(30)

"FreqDist({'campus': 92, 'ifg': 17, 'instituto': 16, 'federal': 15, 'goias': 15, 'educacao': 6, 'secretaria': 5, 'municipal': 5, 'go': 2, 'institutofederal': 1, 'pesquisa': 1, 'planejamento': 1, 'urbano': 1, 'unidade': 1, 'campi': 1})"

In [488]:
aparecida_indexes = get_notna_orgao_normalizado_indexes('aparecida de goiania')

projetos_pesquisa.loc[aparecida_indexes] = projetos_pesquisa.loc[aparecida_indexes].apply(
    link_orgao_to_campus,
    args=('APA', ['unidade'], ['urbano']),
    axis=1
)

In [489]:
temp = projetos_pesquisa.loc[aparecida_indexes]

temp[temp['sigla_orgao'].notna()]['orgao_normalizado'].unique()

array(['instituto federal de goias campus de aparecida de goiania',
       'campus aparecida de goiania', 'ifg campus aparecida de goiania',
       'institutofederal de goias campus aparecida de goiania',
       'instituto federal de goias campus aparecida de goiania go',
       'instituto federal de goias ifg campus aparecida de goiania',
       'unidade aparecida de goiania',
       'instituto federal de goias aparecida de goiania',
       'instituto federal de goias campus aparecida de goiania',
       'campus de aparecida de goiania',
       'instituto federal de educacao campi aparecida de goiania'],
      dtype=object)

#### Goiânia Oeste

In [490]:
corpus_goiania_oeste = get_orgao_normalizado_corpus('goiania oeste')

In [491]:
bag_goiania_oeste = make_bag_from_corpus(corpus_goiania_oeste, ['goiania', 'oeste'])

In [492]:
bag_goiania_oeste.pformat(30)

"FreqDist({'campus': 68, 'instituto': 19, 'federal': 19, 'goias': 17, 'ifg': 11, 'aguas': 1, 'lindas': 1})"

In [493]:
goiania_oeste_indexes = get_notna_orgao_normalizado_indexes('goiania oeste')

projetos_pesquisa.loc[goiania_oeste_indexes] = projetos_pesquisa.loc[goiania_oeste_indexes].apply(
    link_orgao_to_campus,
    args=('OES',),
    axis=1
)

In [494]:
temp = projetos_pesquisa.loc[goiania_oeste_indexes]

temp[temp['sigla_orgao'].notna()]['orgao_normalizado'].unique()

array(['campus goiania oeste', 'ifg campus goiania oeste',
       'instituto federal de goias campus goiania oeste',
       'ifg goiania oeste', 'instituto federal campus goiania oeste',
       'campus aguas lindas e goiania oeste', 'campus goiania oeste ifg'],
      dtype=object)

#### Goiânia

In [495]:
corpus_goiania = get_orgao_normalizado_corpus('goiania')

In [496]:
bag_goiania = make_bag_from_corpus(corpus_goiania, 'goiania')

In [497]:
bag_goiania.pformat(30)

"FreqDist({'campus': 558, 'instituto': 274, 'federal': 273, 'goias': 271, 'aparecida': 108, 'oeste': 74, 'ifg': 39, 'departamento': 7, 'municipal': 6, 'coordenacao': 6, 'educacao': 6, 'secretaria': 5, 'areas': 4, 'academicas': 4, 'turismo': 4, 'hospitalidade': 4, 'matematica': 3, 'daa': 3, 'ii': 3, 'oste': 2, 'laranjeiras': 2, 'pesquisa': 2, 'inhumas': 2, 'jatai': 2, 'go': 2, 'unidade': 2, 'ap': 2, 'colegio': 1, 'marista': 1, 'camara': 1, ...})"

In [498]:
goiania_indexes = get_notna_orgao_normalizado_indexes('goiania')

projetos_pesquisa.loc[goiania_indexes] = projetos_pesquisa.loc[goiania_indexes].apply(
    link_orgao_to_campus,
    args=('GYN', None, ['oeste', 'aparecida', 'ap']),
    axis=1
)

In [499]:
temp = projetos_pesquisa.loc[goiania_indexes]

temp[temp['sigla_orgao'].notna()]['orgao_normalizado'].unique()

array(['instituto federal de goias campus goiania', 'campus goiania',
       'ifg goiania', 'campus goiania dept areas academicas i',
       'campus de goiania', 'ifg campus goiania',
       'coordenacao de turismo e hospitalidade do campus goiania',
       'pro reitoria de pesquisa e pos graduacao campus goiania',
       'campus goiania campus inhumas campus jatai',
       'departamento de turismo e hospitalidade campus goiania',
       'campus goiania ifg', 'campus goiania departamento',
       'campus inhumas goiania e jatai',
       'departamento de areas academicas ifg campus goiania',
       'coordenacao de matematica daa ii campus goiania',
       'instituto federal de goias campus de goiania'], dtype=object)

#### Reitoria

In [500]:
corpus_reitoria = get_orgao_normalizado_corpus('reitoria')

In [501]:
bag_reitoria = make_bag_from_corpus(corpus_reitoria, 'reitoria')

In [502]:
bag_reitoria.pformat(30)

"FreqDist({'pro': 127, 'graduacao': 112, 'pos': 110, 'pesquisa': 109, 'ifg': 19, 'proppg': 5, 'propg': 5, 'ensino': 3, 'extensao': 3, 'educacao': 2, 'assuntos': 2, 'estudantis': 2, 'exteensao': 2, 'ifsp': 2, 'instituto': 1, 'federal': 1, 'ciencia': 1, 'tecnologia': 1, 'goias': 1, 'prppg': 1, 'campus': 1, 'goiania': 1, 'cultura': 1, 'proex': 1, 'graducacao': 1, 'proreitoria': 1, 'ufg': 1, 'inovacao': 1, 'vice': 1, 'vpg': 1})"

In [503]:
reitoria_indexes = get_notna_orgao_normalizado_indexes('reitoria')

projetos_pesquisa.loc[reitoria_indexes] = projetos_pesquisa.loc[reitoria_indexes].apply(
    link_orgao_to_campus,
    args=('REI', None, ['pro']),
    axis=1
)

In [504]:
temp = projetos_pesquisa.loc[reitoria_indexes]

temp[temp['sigla_orgao'].notna()]['orgao_normalizado'].unique()

array(['reitoria ifg',
       'instituto federal de educacao ciencia e tecnologia de goias reitoria',
       'reitoria do ifg', 'ifg reitoria'], dtype=object)

#### Lidar com projetos que tenham mais de um órgão do IFG

In [505]:
def find_more_than_one_campus_reference(row):
    campus_reference_tokens = ['anapolis', 'inhumas', 'goiania', 'luziania', 'itumbiara', 'reitoria', 
                               'lindas', 'canedo', 'uruacu', 'jatai', 'formosa', 'valparaiso', 'cidade']
    match_results = []

    for token in campus_reference_tokens:
        if token in row['orgao_normalizado']:
            match_results.append(True)

    results = list(filter(lambda match_result: match_result is True, match_results))

    if len(results) >= 2:
        print((row['codigo'], row['orgao_normalizado']))

In [506]:
projetos_pesquisa[projetos_pesquisa['orgao_normalizado'].notna()].apply(find_more_than_one_campus_reference, axis=1)

(1280, 'pro reitoria de pesquisa e pos graduacao campus goiania')
(1305, 'campus goiania campus inhumas campus jatai')
(3154, 'campus inhumas goiania e jatai')
(3453, 'campus aguas lindas e goiania oeste')


0       None
1       None
2       None
3       None
4       None
        ... 
4123    None
4124    None
4125    None
4127    None
4128    None
Length: 4127, dtype: object

In [507]:
multicampus_indexes = projetos_pesquisa[projetos_pesquisa['codigo'].isin([1305, 3154, 3453])].index

projetos_pesquisa[projetos_pesquisa['codigo'].isin([1305, 3154, 3453])]

,codigo,nome,descricao,ano_inicio,ano_termino,financiador_projeto,instituicao_financiadora,orgao,codigo_curriculo,sigla_orgao,orgao_normalizado
1292,1305,Alfabetização informacional na universidade: i...,A pesquisa objetiva conhecer como ocorre a imp...,2018,<NA>,NaN,NaN,Câmpus Goiânia; Câmpus Inhumas; Câmpus Jataí,627,GYN,campus goiania campus inhumas campus jatai
3129,3154,Projetos político-pedagógicos das licenciatura...,Face às novas diretrizes curriculares nacionai...,2013,2016,BOLSA-BOLSA-AUXILIO_FINANCEIRO-BOLSA,Conselho Nacional de Desenvolvimento Científic...,"câmpus Inhumas, Goiânia e Jataí",1459,GYN,campus inhumas goiania e jatai
3436,3453,Conhecimento dos estudantesdo Curso Técnico em...,Resumo: Objetivo: Investigar o conhecimento do...,2017,<NA>,NaN,NaN,Câmpus Águas Lindas e Goiânia-oeste,1598,None,campus aguas lindas e goiania oeste


In [508]:
codigo_to_sigla_campus = {
    1305: 'GYN,INH,JAT',
    3154: 'GYN,INH,JAT',
    3453: 'LIN,OES'
}

for index in multicampus_indexes:
    projetos_pesquisa.loc[index, 'sigla_orgao'] = codigo_to_sigla_campus[projetos_pesquisa.loc[index]['codigo']]

In [509]:
projetos_pesquisa[projetos_pesquisa['codigo'].isin([1305, 3154, 3453])].loc[:, ['codigo', 'sigla_orgao']]

,codigo,sigla_orgao
1292,1305,"GYN,INH,JAT"
3129,3154,"GYN,INH,JAT"
3436,3453,"LIN,OES"


## Exportar

In [510]:
export_path = Path(IFG_PRODUZ_STORAGE_DIR / "preprocessed")

if not export_path.is_dir():
    Path(export_path).mkdir()

projetos_pesquisa.to_csv(export_path / "projetos_pesquisa.csv", sep=";", index=False)